# InSituPy demonstration - Add annotations

In [1]:
## The following code ensures that all functions and init files are reloaded before executions.
%load_ext autoreload
%autoreload 2

In [2]:
from pathlib import Path
from insitupy import read_xenium

## Previous steps

1. Download the example data for demonstration: [01_InSituPy_demo_download_data.ipynb](./01_InSituPy_demo_download_data.ipynb)
2. Register images from external stainings: [02_InSituPy_demo_register_images.ipynb](./02_InSituPy_demo_register_images.ipynb)
3. Visualize data with napari and do preprocessing steps: [03_InSituPy_demo_analyze.ipynb](./03_InSituPy_demo_analyze.ipynb)

At this point, the structure of the data should look like this:

    ```
    ./demo_dataset
    ├───cropped_processed
    ├───output-XETG00000__slide_id__sample_id
    │   ├───analysis
    │   │   ├───clustering
    │   │   ├───diffexp
    │   │   ├───pca
    │   │   ├───tsne
    │   │   └───umap
    │   └───cell_feature_matrix
    ├───registered_images
    ├───registration_qc
    └───unregistered_images
    ```


## Load Xenium data into `XeniumData` object

Now the Xenium data can be parsed by providing the data path to `XeniumData`

In [3]:
insitupy_project = Path("demo_dataset/demo_insitupy_project")

In [4]:
xd = read_xenium(insitupy_project)

In [5]:
xd

InSituData
Method:		Xenium
Slide ID:	0001879
Sample ID:	Replicate 1
Path:		C:\Users\ge37voy\Github\InSituPy\notebooks\demo_dataset\demo_insitupy_project
Metadata file:	.ispy

In [6]:
xd.load_images()
xd.load_cells()

Loading images...
Loading cells...
	No alternative cells found...


Note: That the `annotations` and `regions` modalities are not found here is expected. Annotations and regions are added in a later step.

In [7]:
xd

InSituData
Method:		Xenium
Slide ID:	0001879
Sample ID:	Replicate 1
Path:		C:\Users\ge37voy\Github\InSituPy\notebooks\demo_dataset\demo_insitupy_project
Metadata file:	.ispy
    ➤ images
       nuclei:	(25778, 35416)
       CD20:	(25778, 35416)
       HER2:	(25778, 35416)
       HE:	(25778, 35416, 3)
    ➤ cells
       matrix
           AnnData object with n_obs × n_vars = 167780 × 313
           obs: 'transcript_counts', 'control_probe_counts', 'control_codeword_counts', 'total_counts', 'cell_area', 'nucleus_area', 'annotation-demo', 'annotation-demo2', 'dist_from_points'
           var: 'gene_ids', 'feature_types', 'genome'
           obsm: 'spatial'
           varm: 'binned_expression'
       boundaries
           BoundariesData object with 2 entries:
               cellular
               nuclear

## Load annotations

For the analysis of spatial transcriptomic datasets the inclusion of annotations from experts of disease pathology is key. Here, we demonstrate how to annotate data in [QuPath](https://qupath.github.io/), export the annotations as `.geojson` file and import them into the `XeniumData` object.

### Create annotations in QuPath

To create annotations in QuPath, follow these steps:

1. Select a annotation tool from the bar on the top left:

<center><img src="./demo_annotations/qupath_annotation_buttons.jpg" width="300"/></center>

2. Add as many annotations as you want and label them by setting classes in the annotation list. Do not forget to press the "Set class" button:

<center><img src="./demo_annotations/qupath_annotation_list.jpg" width="350"/></center>

3. Export annotations using `File > Export objects as GeoJSON`. Tick `Pretty JSON` to get an easily readable JSON file. The file name needs to have following structure: `annotation-{slide_id}__{sample_id}__{annotation_label}`.

### Import annotations into `XeniumData`

For demonstration purposes, we created a dummy annotation file in `./demo_annotations/`. To add the annotations to `XeniumData` follow the steps below.



In [8]:
xd.import_annotations(
    files=[
        "./demo_annotations/annotations-0001879__Replicate 1__demo.geojson",
        "./demo_annotations/annotations-0001879__Replicate 1__demo2.geojson",
        "./demo_annotations/annotations-mixed_types.geojson"
           ],
    keys=["demo", "demo2", "demo3"]
    )

Importing annotations...


In [9]:
xd.annotations.demo

,objectType,geometry,name,color,origin,scale,layer_type
id,,,,,,,
bd3aacca-1716-4df8-91dd-bf8f6413a7bd,annotation,"POLYGON ((8863.00000 10814.00000, 8863.00000 1...",Positive,"[250, 62, 62]",file,"(0.2125, 0.2125)",Shapes
69814505-4059-42cd-8df2-752f7eb0810d,annotation,"POLYGON ((13096.00000 12492.00000, 13072.40000...",Positive,"[250, 62, 62]",file,"(0.2125, 0.2125)",Shapes
1957cd32-0a21-4b45-9dae-ecf236217140,annotation,"POLYGON ((30975.26000 22938.00000, 30982.00000...",Negative,"[112, 112, 225]",file,"(0.2125, 0.2125)",Shapes
19d2197a-1b8e-456f-8223-fba74641ac1c,annotation,"POLYGON ((31165.00000 16408.00000, 31149.00000...",Negative,"[112, 112, 225]",file,"(0.2125, 0.2125)",Shapes


In [10]:
xd.annotations.demo2

,objectType,geometry,name,color,origin,scale,layer_type
id,,,,,,,
1970eccb-ad38-4b4b-b7a8-54509027b57d,annotation,"POLYGON ((25319.00000 3892.00000, 25313.00000 ...",Negative,"[112, 112, 225]",file,"(0.2125, 0.2125)",Shapes
a3b32cce-1bb9-4a6f-b1d1-9e0c44420cfa,annotation,"POLYGON ((30950.00000 10855.00000, 30944.00000...",Positive,"[250, 62, 62]",file,"(0.2125, 0.2125)",Shapes
92bfe928-a21f-4864-b7cb-f0d300113d88,annotation,"MULTIPOLYGON (((21534.00000 19541.00000, 21534...",Other,"[255, 200, 0]",file,"(0.2125, 0.2125)",Shapes
a6c17a54-6839-40b2-8531-c9227635f344,annotation,"POLYGON ((6501.00000 17126.00000, 6495.00000 1...",Other,"[255, 200, 0]",file,"(0.2125, 0.2125)",Shapes
e78efe2f-d185-4ab6-9cc9-6621897f3662,annotation,"POLYGON ((29519.63000 18523.00000, 29476.00000...",Negative,"[112, 112, 225]",file,"(0.2125, 0.2125)",Shapes


In [11]:
xd.annotations.demo3

,objectType,geometry,name,color,origin,scale,layer_type
id,,,,,,,
8f57c3c3-2216-48b7-99bd-aba12d8c3c41,annotation,"POLYGON ((18016.00000 10643.50000, 18013.24000...",Stroma,"[150, 200, 150]",file,"(0.2125, 0.2125)",Shapes
7e8f8db4-81d4-472e-8e93-0fc756df87aa,annotation,"POLYGON ((12322.00000 6758.00000, 12320.34000 ...",Stroma,"[150, 200, 150]",file,"(0.2125, 0.2125)",Shapes
38a48ddb-f33c-4c61-b996-330b25d84081,annotation,"LINESTRING (16943.80000 7758.84000, 18249.5800...",Necrosis,"[50, 50, 50]",file,"(0.2125, 0.2125)",Shapes
eee244c9-e919-41ae-bb91-44c7abcc0cec,annotation,"LINESTRING (11687.64000 6279.51000, 13439.6900...",Immune cells,"[160, 90, 160]",file,"(0.2125, 0.2125)",Shapes
e3d4c0b6-0998-4692-ab7d-f580f713e275,annotation,POINT (23982.29000 7682.65000),unclassified,"[0, 0, 0]",file,"(0.2125, 0.2125)",Points
e9105240-3b35-489e-994f-e8f9c4786516,annotation,"MULTIPOINT (19857.20000 7768.09000, 20056.2200...",Stroma,"[150, 200, 150]",file,"(0.2125, 0.2125)",Points
2802df97-78ad-44ac-8e6b-d9b9406c8e3f,annotation,"MULTIPOINT (15871.96000 9437.25000, 16611.9600...",Tumor,"[200, 0, 0]",file,"(0.2125, 0.2125)",Points


## Load regions

Regions can be created in QuPath either as described above or using tools like the TMA dearrayer. They are also exported as objects as annotations but different to annotations they do not have a classification and each name of a region has to be unique.

In the following demo regions are read. One of the region files has non-unique names to demonstrate the warning that appears in this case.

In [12]:
from insitupy.io import parse_geopandas
from shapely import Polygon, MultiPolygon

In [13]:
f = "./demo_annotations/annotations-mixed_types.geojson"
df = parse_geopandas(f)

In [14]:
[isinstance(elem, Polygon) for elem in df["geometry"]]

[True, True, False, False, False, False, False]

In [15]:
xd.import_regions(
    files=[
        "./demo_dataset/qupath_project/annotations.geojson"
        ],
    keys=['test'])

Importing regions...


C:\Users\ge37voy\Github\InSituPy\insitupy\_core\dataclasses.py:122: UserWarning: The names of the regions for key 'test' were not unique and thus the key was skipped. In regions only one geometry per class is allowed.
  warnings.warn(
C:\Users\ge37voy\Github\InSituPy\insitupy\_core\insitudata.py:725: UserWarning: Some regions were not pure Polygon objects and skipped.
  self.regions.add_data(data=file,


In [16]:
xd

InSituData
Method:		Xenium
Slide ID:	0001879
Sample ID:	Replicate 1
Path:		C:\Users\ge37voy\Github\InSituPy\notebooks\demo_dataset\demo_insitupy_project
Metadata file:	.ispy
    ➤ images
       nuclei:	(25778, 35416)
       CD20:	(25778, 35416)
       HER2:	(25778, 35416)
       HE:	(25778, 35416, 3)
    ➤ cells
       matrix
           AnnData object with n_obs × n_vars = 167780 × 313
           obs: 'transcript_counts', 'control_probe_counts', 'control_codeword_counts', 'total_counts', 'cell_area', 'nucleus_area', 'annotation-demo', 'annotation-demo2', 'dist_from_points'
           var: 'gene_ids', 'feature_types', 'genome'
           obsm: 'spatial'
           varm: 'binned_expression'
       boundaries
           BoundariesData object with 2 entries:
               cellular
               nuclear
    ➤ annotations
       demo:	4 annotations, 2 classes ('Positive','Negative') 
       demo2:	5 annotations, 3 classes ('Negative','Positive','Other') 
       demo3:	7 annotations, 5 clas

In [17]:
xd.import_regions(
    files=[
        "./demo_regions/regions-0001879__Replicate 1__demo_regions.geojson",
        "./demo_regions/regions-0001879__Replicate 1__TMA.geojson",
        ],
    keys=['demo_regions', 'TMA'])

Importing regions...


In [18]:
xd

InSituData
Method:		Xenium
Slide ID:	0001879
Sample ID:	Replicate 1
Path:		C:\Users\ge37voy\Github\InSituPy\notebooks\demo_dataset\demo_insitupy_project
Metadata file:	.ispy
    ➤ images
       nuclei:	(25778, 35416)
       CD20:	(25778, 35416)
       HER2:	(25778, 35416)
       HE:	(25778, 35416, 3)
    ➤ cells
       matrix
           AnnData object with n_obs × n_vars = 167780 × 313
           obs: 'transcript_counts', 'control_probe_counts', 'control_codeword_counts', 'total_counts', 'cell_area', 'nucleus_area', 'annotation-demo', 'annotation-demo2', 'dist_from_points'
           var: 'gene_ids', 'feature_types', 'genome'
           obsm: 'spatial'
           varm: 'binned_expression'
       boundaries
           BoundariesData object with 2 entries:
               cellular
               nuclear
    ➤ annotations
       demo:	4 annotations, 2 classes ('Positive','Negative') 
       demo2:	5 annotations, 3 classes ('Negative','Positive','Other') 
       demo3:	7 annotations, 5 clas

In [19]:
xd.import_regions(
    files=[
        "./demo_dataset/qupath_project/annotations.geojson"
        ],
    keys=['test'])

Importing regions...


C:\Users\ge37voy\Github\InSituPy\insitupy\_core\dataclasses.py:122: UserWarning: The names of the regions for key 'test' were not unique and thus the key was skipped. In regions only one geometry per class is allowed.
  warnings.warn(
C:\Users\ge37voy\Github\InSituPy\insitupy\_core\insitudata.py:725: UserWarning: Some regions were not pure Polygon objects and skipped.
  self.regions.add_data(data=file,


In [20]:
xd

InSituData
Method:		Xenium
Slide ID:	0001879
Sample ID:	Replicate 1
Path:		C:\Users\ge37voy\Github\InSituPy\notebooks\demo_dataset\demo_insitupy_project
Metadata file:	.ispy
    ➤ images
       nuclei:	(25778, 35416)
       CD20:	(25778, 35416)
       HER2:	(25778, 35416)
       HE:	(25778, 35416, 3)
    ➤ cells
       matrix
           AnnData object with n_obs × n_vars = 167780 × 313
           obs: 'transcript_counts', 'control_probe_counts', 'control_codeword_counts', 'total_counts', 'cell_area', 'nucleus_area', 'annotation-demo', 'annotation-demo2', 'dist_from_points'
           var: 'gene_ids', 'feature_types', 'genome'
           obsm: 'spatial'
           varm: 'binned_expression'
       boundaries
           BoundariesData object with 2 entries:
               cellular
               nuclear
    ➤ annotations
       demo:	4 annotations, 2 classes ('Positive','Negative') 
       demo2:	5 annotations, 3 classes ('Negative','Positive','Other') 
       demo3:	7 annotations, 5 clas

Properties of the added `anotations` and `regions` can be inspected in the XeniumData representation:

In [21]:
xd.show()

Invalid schema for package 'ome-types', please run 'npe2 validate ome-types' to check for manifest errors.


In [14]:
xd.store_geometries()

Names of regions for key 'TestKey' are unique.
Added 1 new regions to key 'TestKey'
Names of regions for key 'TestKey' are unique.
Added 1 new regions to existing key 'TestKey'


In [15]:
xd

InSituData
Method:		Xenium
Slide ID:	0001879
Sample ID:	Replicate 1
Path:		C:\Users\ge37voy\Github\InSituPy\notebooks\demo_dataset\demo_insitupy_project
Metadata file:	.ispy
    ➤ images
       nuclei:	(25778, 35416)
       CD20:	(25778, 35416)
       HER2:	(25778, 35416)
       HE:	(25778, 35416, 3)
    ➤ cells
       matrix
           AnnData object with n_obs × n_vars = 167780 × 313
           obs: 'transcript_counts', 'control_probe_counts', 'control_codeword_counts', 'total_counts', 'cell_area', 'nucleus_area', 'annotation-demo', 'annotation-demo2', 'dist_from_points'
           var: 'gene_ids', 'feature_types', 'genome'
           obsm: 'spatial'
           varm: 'binned_expression'
       boundaries
           BoundariesData object with 2 entries:
               cellular
               nuclear
    ➤ annotations
       demo:	4 annotations, 2 classes ('Positive','Negative') 
       demo2:	5 annotations, 3 classes ('Negative','Positive','Other') 
    ➤ regions
       demo_regions:	

In [72]:
xd.annotations.demo

,objectType,geometry,name,color,origin,scale,layer_type
id,,,,,,,
bd3aacca-1716-4df8-91dd-bf8f6413a7bd,annotation,"POLYGON ((8863.00000 10814.00000, 8863.00000 1...",Positive,"[250, 62, 62]",file,"(0.2125, 0.2125)",Shapes
69814505-4059-42cd-8df2-752f7eb0810d,annotation,"POLYGON ((13096.00000 12492.00000, 13072.40000...",Positive,"[250, 62, 62]",file,"(0.2125, 0.2125)",Shapes
1957cd32-0a21-4b45-9dae-ecf236217140,annotation,"POLYGON ((30975.26000 22938.00000, 30982.00000...",Negative,"[112, 112, 225]",file,"(0.2125, 0.2125)",Shapes
19d2197a-1b8e-456f-8223-fba74641ac1c,annotation,"POLYGON ((31165.00000 16408.00000, 31149.00000...",Negative,"[112, 112, 225]",file,"(0.2125, 0.2125)",Shapes


In [73]:
xd.annotations.demo3

,objectType,geometry,name,color,origin,scale,layer_type
id,,,,,,,
8f57c3c3-2216-48b7-99bd-aba12d8c3c41,annotation,"POLYGON ((18016.00000 10643.50000, 18013.24000...",Stroma,"[150, 200, 150]",file,"(0.2125, 0.2125)",Shapes
7e8f8db4-81d4-472e-8e93-0fc756df87aa,annotation,"POLYGON ((12322.00000 6758.00000, 12320.34000 ...",Stroma,"[150, 200, 150]",file,"(0.2125, 0.2125)",Shapes
38a48ddb-f33c-4c61-b996-330b25d84081,annotation,"LINESTRING (16943.80000 7758.84000, 18249.5800...",Necrosis,"[50, 50, 50]",file,"(0.2125, 0.2125)",Shapes
eee244c9-e919-41ae-bb91-44c7abcc0cec,annotation,"LINESTRING (11687.64000 6279.51000, 13439.6900...",Immune cells,"[160, 90, 160]",file,"(0.2125, 0.2125)",Shapes
e3d4c0b6-0998-4692-ab7d-f580f713e275,annotation,POINT (23982.29000 7682.65000),unclassified,"[0, 0, 0]",file,"(0.2125, 0.2125)",Points
e9105240-3b35-489e-994f-e8f9c4786516,annotation,"MULTIPOINT (19857.20000 7768.09000, 20056.2200...",Stroma,"[150, 200, 150]",file,"(0.2125, 0.2125)",Points
2802df97-78ad-44ac-8e6b-d9b9406c8e3f,annotation,"MULTIPOINT (15871.96000 9437.25000, 16611.9600...",Tumor,"[200, 0, 0]",file,"(0.2125, 0.2125)",Points


In [74]:
xd.show()

In [16]:
xd.regions.TestKey

,objectType,geometry,name,color,origin,scale,layer_type
id,,,,,,,
49d9c3b8-2d68-4341-87e8-ef834d20f436,annotation,"POLYGON ((9911.09323 9662.87386, 9362.05054 10...",TestClass,"[255, 170, 0]",manual,"[0.2125, 0.2125]",Shapes
f190a3b9-834c-41c3-a81a-7a7b7c56fd46,annotation,"POLYGON ((11668.02984 6780.39973, 11668.02984 ...",TestClass,"[255, 170, 0]",manual,"[0.2125, 0.2125]",Shapes
f43057dc-6390-4fcd-90fe-8615b9fcdbaa,annotation,"LINESTRING (8346.32156 7384.34669, 11036.63075...",TestClass,"[255, 170, 0]",manual,"[0.2125, 0.2125]",Shapes
3f0edd7d-65aa-431b-979a-054ec2768150,annotation,"POLYGON ((8895.36425 9086.37903, 8887.69726 89...",TestClass,"[255, 170, 0]",manual,"[0.2125, 0.2125]",Shapes
e1838c34-034d-4334-b1d4-e12dad7c685b,annotation,"POLYGON ((22511.62300 12710.06080, 22499.32937...",TestClass,"[255, 170, 0]",manual,"[0.2125, 0.2125]",Shapes


In [17]:
xd.regions.TMA

,objectType,name,isMissing,geometry,origin,scale,layer_type
id,,,,,,,
7ab5d5a6-49bd-4122-bc64-05477bc0207b,tmaCore,B-2,False,"POLYGON ((20233.46000 19826.77000, 20228.82000...",file,"(0.2125, 0.2125)",Shapes
06ef93c1-f86d-45e6-ad9a-896e254638ea,tmaCore,A-3,False,"POLYGON ((33887.96000 4250.67000, 33883.32000 ...",file,"(0.2125, 0.2125)",Shapes
7933d3fd-ccd3-46af-8f15-fcc01ec9c128,tmaCore,B-1,False,"POLYGON ((7318.34000 20393.63000, 7313.69000 2...",file,"(0.2125, 0.2125)",Shapes
7015118d-2947-48e3-baf0-4b220a76a951,tmaCore,B-3,False,"POLYGON ((34405.52000 19900.71000, 34400.88000...",file,"(0.2125, 0.2125)",Shapes
bf86657f-31f6-40fe-983b-f80c3d75512b,tmaCore,A-1,False,"POLYGON ((6973.30000 4275.31000, 6968.65000 44...",file,"(0.2125, 0.2125)",Shapes
440d8f00-fb0e-42e7-9f98-30d30adfc8df,tmaCore,A-2,False,"POLYGON ((19641.96000 4028.85000, 19637.32000 ...",file,"(0.2125, 0.2125)",Shapes


In [24]:
xd.store_geometries()

uniqueness?
True
uniqueness?
True
uniqueness?
True
uniqueness?
True
                                     objectType  \
id                                                
8fc203cc-379b-4e87-8826-66b57d0c0b45     region   
1d629d7d-2f16-4e52-b2c9-9b7b2a26dc63     region   
7ef5438f-5050-46e2-9d0c-c7e485e40815     region   

                                                                               geometry  \
id                                                                                        
8fc203cc-379b-4e87-8826-66b57d0c0b45  POLYGON ((16527.05766 6396.06984, 16170.17991 ...   
1d629d7d-2f16-4e52-b2c9-9b7b2a26dc63  POLYGON ((27507.91149 8098.10219, 27041.22521 ...   
7ef5438f-5050-46e2-9d0c-c7e485e40815  POLYGON ((24625.43736 13259.10349, 24268.55961...   

                                         name          color  origin  \
id                                                                     
8fc203cc-379b-4e87-8826-66b57d0c0b45  asdfsff  [255, 170, 0]  manual   
1d62

C:\Users\ge37voy\Github\InSituPy\insitupy\_core\dataclasses.py:114: UserWarning: Names of regions for key 'dsffsdf' were not unique. Key was skipped.
  warnings.warn(message=f"Names of {self.shape_name} for key '{key}' were not unique. Key was skipped.")


In [25]:
xd

InSituData
Method:		Xenium
Slide ID:	0001879
Sample ID:	Replicate 1
Path:		C:\Users\ge37voy\Github\InSituPy\notebooks\demo_dataset\demo_insitupy_project
Metadata file:	.ispy
    ➤ images
       nuclei:	(25778, 35416)
       CD20:	(25778, 35416)
       HER2:	(25778, 35416)
       HE:	(25778, 35416, 3)
    ➤ cells
       matrix
           AnnData object with n_obs × n_vars = 167780 × 313
           obs: 'transcript_counts', 'control_probe_counts', 'control_codeword_counts', 'total_counts', 'cell_area', 'nucleus_area', 'annotation-demo', 'annotation-demo2', 'dist_from_points'
           var: 'gene_ids', 'feature_types', 'genome'
           obsm: 'spatial'
           varm: 'binned_expression'
       boundaries
           BoundariesData object with 2 entries:
               cellular
               nuclear
    ➤ annotations
       demo:	4 annotations, 2 classes ('Positive','Negative') 
       demo2:	5 annotations, 3 classes ('Negative','Positive','Other') 
       TestKey:	7 annotations, 1 cl

In [22]:
type(xd.regions)

insitupy._core.dataclasses.RegionsData

In [15]:
xd.show()

# Lines should not be allowed for Regions

In [18]:
xd.viewer.layers["nuclei"].name

'nuclei'

In [14]:
xd

InSituData
Method:		Xenium
Slide ID:	0001879
Sample ID:	Replicate 1
Path:		C:\Users\ge37voy\Github\InSituPy\notebooks\demo_dataset\demo_insitupy_project
Metadata file:	.ispy
    ➤ images
       nuclei:	(25778, 35416)
       CD20:	(25778, 35416)
       HER2:	(25778, 35416)
       HE:	(25778, 35416, 3)
    ➤ cells
       matrix
           AnnData object with n_obs × n_vars = 167780 × 313
           obs: 'transcript_counts', 'control_probe_counts', 'control_codeword_counts', 'total_counts', 'cell_area', 'nucleus_area', 'annotation-demo', 'annotation-demo2', 'dist_from_points'
           var: 'gene_ids', 'feature_types', 'genome'
           obsm: 'spatial'
           varm: 'binned_expression'
       boundaries
           BoundariesData object with 2 entries:
               cellular
               nuclear
    ➤ annotations
       demo:	4 annotations, 2 classes ('Positive','Negative') 
       demo2:	5 annotations, 3 classes ('Negative','Positive','Other') 
       TestKey:	6 annotations, 1 cl

### Visualize and edit annotations and regions using napari

To visualize annotations and regions in napari, three widgets are available:
- "Add annotations"
- "Show regions"
- "Show annotations"

The widgets "Show annotations" and "Show regions" are grouped together and accessible via tabs:

<left><img src="./demo_annotations/napari_region+annotations_widget.jpg" width="300"/></left>


Using thes widgets, annotations and regions can be displayed. New annotations can be added using the "Add annotations" widget.

Annotations can also be displayed while starting the napari viewer with `.show()` using the `annotation_keys` argument:


In [13]:
xd.show(annotation_keys="all")

#### Annotation layers

The annotations are added as shapes layers to the layer list. The layer name always starts with a "*" and has following syntax: `"* Class (Label)"`:

<left><img src="./demo_annotations/napari_layerlist_annotations.jpg" width="300"/></left>

- **Label**: A label for one collection of annotations. Could e.g. tell us who did the annotations or what is the focus of this collection of annotations.
- **Class**: Specifies the class of one specific annotation. Could be e.g. the name of cells, the morphological structure or the disease state annotated.

#### Add custom annotations using the Annotation Widget

<left><img src="./demo_annotations/napari_annotation_widget.jpg" width="200"/></left>

By clicking the `"Add annotation layer"` button a new layer with the above mentioned syntax is added. The layer controls on the top left can be then used to add new shapes as annotations:

<left><img src="./demo_annotations/napari_layerconrols_annotations.jpg" width="300"/></left>

An example annotation is shown here:

<left><img src="./demo_annotations/napari_annotation_example.jpg" width="200"/></left>

The annotations can then be stored in the `XeniumData` object using the `store_annotations` function.


In [15]:
xd.store_annotations()

Added 3 new annotations to existing key 'demo2'


In [16]:
xd

XeniumData
Slide ID:	0001879
Sample ID:	Replicate 1
Data path:	demo_dataset
Data folder:	demo_insitupy_project
Metadata file:	.ispy
    ➤ images
       nuclei:	(25778, 35416)
       CD20:	(25778, 35416)
       HER2:	(25778, 35416)
       HE:	(25778, 35416, 3)
    ➤ cells
       matrix
           AnnData object with n_obs × n_vars = 167780 × 313
           obs: 'transcript_counts', 'control_probe_counts', 'control_codeword_counts', 'total_counts', 'cell_area', 'nucleus_area'
           var: 'gene_ids', 'feature_types', 'genome'
           obsm: 'spatial'
           varm: 'binned_expression'
       boundaries
           BoundariesData object with 2 entries:
               cellular
               nuclear
    ➤ annotations
       demo:	4 annotations, 2 classes ('Positive','Negative') 
       demo2:	7 annotations, 4 classes ('Negative','Positive','Other','test_class') 
    ➤ regions
       demo_regions:	3 regions, 3 classes ('Region1','Region2','Region3') 
       TMA:	6 regions, 6 classes (

### Assign annotations to observations

To use the annotations in analyses (e.g. to select only observations within a certain annotation or compare gene expression between different annotations) one can use the `assign_annotations` function. It adds columns containing the annotation class to `xd.matrix.obs`. The column has the syntax `annotation-{Label}` and if an observation is not part of any annotation within this label, it contains `NaN`. 

In [17]:
xd.assign_annotations()

Assigning key 'demo'...
Assigning key 'demo2'...


After assigning the annotations, the labels analyzed here are marked with a ✔:

In [18]:
xd

XeniumData
Slide ID:	0001879
Sample ID:	Replicate 1
Data path:	demo_dataset
Data folder:	demo_insitupy_project
Metadata file:	.ispy
    ➤ images
       nuclei:	(25778, 35416)
       CD20:	(25778, 35416)
       HER2:	(25778, 35416)
       HE:	(25778, 35416, 3)
    ➤ cells
       matrix
           AnnData object with n_obs × n_vars = 167780 × 313
           obs: 'transcript_counts', 'control_probe_counts', 'control_codeword_counts', 'total_counts', 'cell_area', 'nucleus_area', 'annotation-demo', 'annotation-demo2'
           var: 'gene_ids', 'feature_types', 'genome'
           obsm: 'spatial'
           varm: 'binned_expression'
       boundaries
           BoundariesData object with 2 entries:
               cellular
               nuclear
    ➤ annotations
       demo:	4 annotations, 2 classes ('Positive','Negative') ✔
       demo2:	7 annotations, 4 classes ('Negative','Positive','Other','test_class') ✔
    ➤ regions
       demo_regions:	3 regions, 3 classes ('Region1','Region2','Regi

Following cells show examples how to explore the assigned annotations:

In [19]:
# print number of cells within one annotation
xd.cells.matrix.obs["annotation-demo2"].notna().sum()

21199

In [20]:
# show only observations that were part of this annotation label
xd.cells.matrix.obs[xd.cells.matrix.obs["annotation-demo2"].notna()]

,transcript_counts,control_probe_counts,control_codeword_counts,total_counts,cell_area,nucleus_area,annotation-demo,annotation-demo2
4921,281,0,0,281,733.247187,26.010000,NaN,Other
4922,273,1,0,274,380.576875,30.074063,NaN,Other
4923,189,2,0,191,285.658437,8.263594,NaN,Other
4924,212,0,0,212,282.226562,24.068281,NaN,Other
4925,58,0,0,58,81.823125,4.470469,NaN,Other
...,...,...,...,...,...,...,...,...
165374,96,1,0,97,150.234844,11.063281,Negative,Negative
165375,379,0,0,379,153.666719,75.681875,Negative,Negative
165376,101,0,0,101,27.996875,17.836719,Negative,Negative
165377,472,0,0,472,200.177656,52.652188,Negative,Negative


## Save imported annotations in `InSituPy` project

In [25]:
xd.save()

Updating project in c:\Users\ge37voy\Github\InSituPy\notebooks\demo_dataset\demo_insitupy_project
	Updating cells...
	Updating annotations...
	Updating regions...
Saved.
Reloading following modalities: annotations,cells,images,regions
Loading annotations...
Loading cells...
	No alternative cells found...
Loading images...
Loading regions...


In [26]:
xd.show()